# Titanic Dataset – Data Cleaning & Feature Engineering

Este proyecto realiza un proceso completo de **limpieza, transformación y análisis de datos**
sobre el dataset del Titanic utilizando **Python y pandas**.

El objetivo es preparar los datos para un análisis posterior, aplicando técnicas habituales
de data cleaning, feature engineering y lógica de negocio.


## Objetivos del proyecto

- Analizar y detectar valores nulos
- Aplicar diferentes estrategias de imputación
- Limpiar columnas de texto mediante expresiones regulares
- Crear nuevas variables (feature engineering)
- Realizar análisis numérico y rankings
- Calcular métricas personalizadas por pasajero


In [4]:
import pandas as pd
from unidecode import unidecode


## Carga del dataset

Cargamos el dataset original del Titanic desde la carpeta `data/`. 
Esto nos permitirá trabajar con los datos en pandas y realizar la limpieza 
y análisis posteriores. Mostramos las primeras filas para comprobar la estructura.


In [9]:
df = pd.read_excel('../data/Titanic.xlsx')
df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Carga del dataset

Hemos cargado el dataset **Titanic.xlsx** desde la carpeta `data/` utilizando **pandas**.  

La función `pd.read_excel()` permite leer archivos de Excel directamente en un DataFrame de pandas.  
El método `head()` nos muestra las primeras cinco filas para verificar que la carga se haya realizado correctamente y que los nombres de las columnas y los tipos de datos son los esperados.


## Comprobación de valores nulos

El primer paso en la limpieza de datos es identificar los valores faltantes (nulos) en el dataset.  

- Creamos un **DataFrame booleano** que indique si cada celda tiene un valor nulo (`True`) o no (`False`).  
- Contamos los **valores nulos por columna** y el **total de valores nulos en todo el DataFrame** para tener una visión general del estado de los datos.


In [10]:
# DataFrame booleano indicando valores nulos
valores_nulos = df.isnull()
valores_nulos.head()

# Total de valores nulos por columna
nulos_por_columna = df.isnull().sum()
print("Valores nulos por columna:\n", nulos_por_columna)

# Total de valores nulos en todo el DataFrame
total_nulos = df.isnull().sum().sum()
print("\nTotal de valores nulos en el DataFrame:", total_nulos)


Valores nulos por columna:
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Total de valores nulos en el DataFrame: 866


### Resultados de la comprobación de valores nulos

Tras ejecutar la comprobación:

- La columna **Age** tiene 177 valores nulos.  
- La columna **Cabin** es la que más valores nulos presenta, con 687 registros faltantes.  
- La columna **Embarked** tiene 2 valores nulos.  
- El total de valores nulos en el DataFrame es de **866**.

Esto nos indica que debemos aplicar estrategias de imputación o limpieza para estas columnas antes de continuar con el análisis.


### Relleno de valores nulos (Imputación)

Aplicaremos diferentes estrategias según la columna:

- **Age:** rellenaremos los valores nulos con la media de la columna.  
- **Fare:** rellenaremos los valores nulos con un valor constante (100).  
- **Embarked:** rellenaremos los valores nulos con la moda.  
- **Cabin:** aplicaremos primero un relleno hacia adelante (ffill) y luego hacia atrás (bfill).


In [12]:
# Relleno de Age con la media
df['Age'].fillna(round(df['Age'].mean(), 0), inplace=True)

# Relleno de Fare con valor constante
df['Fare'].fillna(100, inplace=True)

# Relleno de Embarked con la moda
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

# Relleno de Cabin hacia adelante y hacia atrás
df['Cabin'].fillna(method='ffill', inplace=True)
df['Cabin'].fillna(method='bfill', inplace=True)

# Comprobación de valores nulos después del relleno
df.isnull().sum()


C:\Users\Usuario\AppData\Local\Temp\ipykernel_11796\4127617805.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(round(df['Age'].mean(), 0), inplace=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_11796\4127617805.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a 

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

### Resultado final del relleno de valores nulos

Después de aplicar las estrategias de imputación:

- **Age:** se reemplazaron los valores nulos por la media de la columna.  
- **Fare:** se reemplazaron los valores nulos por 100.  
- **Embarked:** se reemplazaron los valores nulos por la moda.  
- **Cabin:** se rellenaron primero hacia adelante (ffill) y luego hacia atrás (bfill).  

Ahora, **ninguna columna tiene valores nulos**, como se observa en el conteo final de valores nulos:

